In [1]:
%load_ext autoreload
%autoreload 2



%autoreload 2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.spatial.distance import pdist, squareform
from scipy.special import expit
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt
from sklearn.cluster import SpectralClustering  # Add this import
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()

# Load the RDS file
readRDS = robjects.r['readRDS']
pce_data = readRDS('/Users/sarahurbut/Library/Cloudstorage/Dropbox-Personal/pce_df_prevent.rds')
pce_df = pandas2ri.rpy2py(pce_data)  # Convert to pandas DataFrame
def load_model_essentials(base_path='/Users/sarahurbut/Library/CloudStorage/Dropbox-Personal/data_for_running/'):
    """
    Load all essential components
    """
    print("Loading components...")
    
    # Load large matrices
    Y = torch.load(base_path + 'Y_tensor.pt')
    E = torch.load(base_path + 'E_matrix.pt')
    G = torch.load(base_path + 'G_matrix.pt')
    
    # Load other components
    essentials = torch.load(base_path + 'model_essentials.pt')
    
    print("Loaded all components successfully!")
    
    return Y, E, G, essentials

# Load and initialize model:
Y, E, G, essentials = load_model_essentials()

from clust_huge_amp import *
# Subset the data

# Subset the data
Y_100k, E_100k, G_100k, indices = subset_data(Y, E, G, start_index=0, end_index=10000)

Loading components...


/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_45444/1308036556.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  Y = torch.load(base_path + 'Y_tensor.pt')
/v

Loaded all components successfully!


# Comparie pis

In [23]:
pi_batch = torch.load("/Users/sarahurbut/Library/CloudStorage/Dropbox-Personal/pi_enroll_age/pi_enroll_age_offset_0_sex_0_10000_try2.pt")

/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_9974/692683130.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pi_batch = torch.load("/Users/sarahurbut/Librar

In [6]:
pi_original=torch.load("/Users/sarahurbut/Library/CloudStorage/Dropbox-Personal/pi_enroll_sex_0_10000.pt")

/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_9974/259286109.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pi_original=torch.load("/Users/sarahurbut/Libra

In [7]:
are_equal = torch.allclose(pi_original, pi_batch, atol=1e-6)
print("Are the interactive and batch predictions the same?", are_equal)

Are the interactive and batch predictions the same? True


In [10]:
from evaluatetdccode import *

import pandas as pd
pce_df=pd.read_csv('/Users/sarahurbut/Library/Cloudstorage/Dropbox-Personal/baselinagefamh.csv')


In [24]:
from evaluatetdccode import *

evaluate_major_diseases_wsex_with_bootstrap_dynamic_1year_different_start_end(
    pi_batch, Y_100k, E_100k, essentials['disease_names'], pce_df, 
        n_bootstraps=20, 
        follow_up_duration_years=10, 
        start_offset=0
    )

Filtering for Female: Found 5409 individuals in cohort
Filtering for Male: Found 4591 individuals in cohort

Summary of Results (1-Year Risk, Sex-Adjusted, Offset=0):
--------------------------------------------------------------------------------
Disease Group        AUC                       Events     Rate (%)   C-index   
--------------------------------------------------------------------------------
ASCVD                0.712 (0.688-0.730)       831        8.3        0.289
Diabetes             0.622 (0.591-0.631)       581        5.8        0.369
Atrial_Fib           0.684 (0.664-0.709)       376        3.8        0.320
CKD                  0.708 (0.669-0.737)       207        2.1        0.285
All_Cancers          0.680 (0.654-0.704)       480        4.8        0.321
Stroke               0.654 (0.620-0.690)       129        1.3        0.348
Heart_Failure        0.698 (0.660-0.720)       205        2.1        0.301
Pneumonia            0.683 (0.647-0.707)       335        3.4     

{'ASCVD': {'auc': 0.7118249781906516,
  'n_events': 831,
  'event_rate': 8.309999999999999,
  'ci_lower': 0.6883009410698732,
  'ci_upper': 0.7303574278856229,
  'c_index': 0.2890123469697416},
 'Diabetes': {'auc': 0.622152937657231,
  'n_events': 581,
  'event_rate': 5.81,
  'ci_lower': 0.5909697439382894,
  'ci_upper': 0.6308646285740446,
  'c_index': 0.3689286498602527},
 'Atrial_Fib': {'auc': 0.6839643188708694,
  'n_events': 376,
  'event_rate': 3.7600000000000002,
  'ci_lower': 0.6635614353279055,
  'ci_upper': 0.7091937512049818,
  'c_index': 0.32022563103644186},
 'CKD': {'auc': 0.7077790455669064,
  'n_events': 207,
  'event_rate': 2.07,
  'ci_lower': 0.6685202810682223,
  'ci_upper': 0.7374554840625825,
  'c_index': 0.28497587059238955},
 'All_Cancers': {'auc': 0.6797875087535014,
  'n_events': 480,
  'event_rate': 4.8,
  'ci_lower': 0.6536963140283448,
  'ci_upper': 0.7041893165531082,
  'c_index': 0.32143741728854747},
 'Stroke': {'auc': 0.6539612944974669,
  'n_events': 12

## now try ten year with a different year offset

In [7]:
from evaluatetdccode import *

pi_batch = torch.load(f"/Users/sarahurbut/Library/CloudStorage/Dropbox-Personal/pi_enroll_fixedphi_age_offset_3_sex_0_10000_try2.pt")
 
evaluate_major_diseases_wsex_with_bootstrap_dynamic_1year_different_start_end(
    pi_batch, Y_100k, E_100k, essentials['disease_names'], pce_df, 
        n_bootstraps=20, 
        follow_up_duration_years=10, 
        start_offset=3
    )


/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_45444/1600016889.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pi_batch = torch.load(f"/Users/sarahurbut/Lib

Filtering for Female: Found 5409 individuals in cohort
Filtering for Male: Found 4591 individuals in cohort

Summary of Results (1-Year Risk, Sex-Adjusted, Offset=3):
--------------------------------------------------------------------------------
Disease Group        AUC                       Events     Rate (%)   C-index   
--------------------------------------------------------------------------------
ASCVD                0.665 (0.646-0.680)       932        9.3        0.332
Diabetes             0.596 (0.580-0.616)       658        6.6        0.398
Atrial_Fib           0.683 (0.667-0.698)       303        3.0        0.319
CKD                  0.708 (0.687-0.730)       361        3.6        0.292
All_Cancers          0.640 (0.619-0.653)       611        6.1        0.354
Stroke               0.677 (0.652-0.711)       212        2.1        0.315
Heart_Failure        0.702 (0.668-0.724)       286        2.9        0.303
Pneumonia            0.668 (0.641-0.681)       472        4.7     

{'ASCVD': {'auc': 0.6652894747553535,
  'n_events': 932,
  'event_rate': 9.32,
  'ci_lower': 0.6464248737667739,
  'ci_upper': 0.6796633546906273,
  'c_index': 0.33212023461791323},
 'Diabetes': {'auc': 0.5960937108551178,
  'n_events': 658,
  'event_rate': 6.58,
  'ci_lower': 0.5799209855103055,
  'ci_upper': 0.6158564177180181,
  'c_index': 0.3981859660296983},
 'Atrial_Fib': {'auc': 0.6826651922732876,
  'n_events': 303,
  'event_rate': 3.0300000000000002,
  'ci_lower': 0.6670188912696368,
  'ci_upper': 0.6976716379305399,
  'c_index': 0.31880577128129933},
 'CKD': {'auc': 0.7081472170277776,
  'n_events': 361,
  'event_rate': 3.61,
  'ci_lower': 0.6866170734242406,
  'ci_upper': 0.7297995419580151,
  'c_index': 0.2920776498836744},
 'All_Cancers': {'auc': 0.6395354350487451,
  'n_events': 611,
  'event_rate': 6.11,
  'ci_lower': 0.618923848614673,
  'ci_upper': 0.6530563304189514,
  'c_index': 0.35388640323598375},
 'Stroke': {'auc': 0.6766822196605778,
  'n_events': 212,
  'event_

In [25]:
import pandas as pd
from evaluatetdccode import evaluate_major_diseases_wsex_with_bootstrap_dynamic_1year_different_start_end


all_results = []
for k in range(10):
    pi_offset = torch.load(f"/Users/sarahurbut/Library/CloudStorage/Dropbox-Personal/pi_enroll_age/pi_enroll_age_offset_{k}_sex_0_10000_try2.pt")
    results = evaluate_major_diseases_wsex_with_bootstrap_dynamic_1year_different_start_end(
        pi_offset, Y_100k, E_100k, essentials['disease_names'], pce_df, 
        n_bootstraps=20, 
        follow_up_duration_years=1, 
        start_offset=k
    )
    df = pd.DataFrame(results).T.reset_index().rename(columns={'index': 'Disease'})
    df['offset'] = k
    all_results.append(df)

/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_9974/1440258676.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pi_offset = torch.load(f"/Users/sarahurbut/Lib

Filtering for Female: Found 5409 individuals in cohort
Filtering for Male: Found 4591 individuals in cohort

Summary of Results (1-Year Risk, Sex-Adjusted, Offset=0):
--------------------------------------------------------------------------------
Disease Group        AUC                       Events     Rate (%)   C-index   
--------------------------------------------------------------------------------
ASCVD                0.856 (0.798-0.903)       78         0.8        N/A
Diabetes             0.732 (0.648-0.831)       33         0.3        N/A
Atrial_Fib           0.794 (0.725-0.879)       22         0.2        N/A
CKD                  0.874 (0.766-0.997)       4          0.0        N/A
All_Cancers          0.765 (0.696-0.831)       26         0.3        N/A
Stroke               0.731 (0.602-0.856)       5          0.1        N/A
Heart_Failure        0.846 (0.705-0.939)       15         0.1        N/A
Pneumonia            0.484 (0.363-0.625)       15         0.1        N/A
COPD   

/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_9974/1440258676.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pi_offset = torch.load(f"/Users/sarahurbut/Lib

Filtering for Female: Found 5409 individuals in cohort
Filtering for Male: Found 4591 individuals in cohort

Summary of Results (1-Year Risk, Sex-Adjusted, Offset=1):
--------------------------------------------------------------------------------
Disease Group        AUC                       Events     Rate (%)   C-index   
--------------------------------------------------------------------------------
ASCVD                0.846 (0.785-0.874)       87         0.9        N/A
Diabetes             0.732 (0.669-0.799)       53         0.5        N/A
Atrial_Fib           0.777 (0.664-0.870)       22         0.2        N/A
CKD                  0.612 (0.469-0.689)       5          0.1        N/A
All_Cancers          0.741 (0.663-0.817)       44         0.4        N/A
Stroke               0.798 (0.619-0.912)       5          0.1        N/A
Heart_Failure        0.664 (0.286-0.947)       5          0.1        N/A
Pneumonia            0.661 (0.544-0.774)       12         0.1        N/A
COPD   

/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_9974/1440258676.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pi_offset = torch.load(f"/Users/sarahurbut/Lib

...

Filtering for Female: Found 5409 individuals in cohort
Filtering for Male: Found 4591 individuals in cohort

Summary of Results (1-Year Risk, Sex-Adjusted, Offset=7):
--------------------------------------------------------------------------------
Disease Group        AUC                       Events     Rate (%)   C-index   
--------------------------------------------------------------------------------
ASCVD                0.877 (0.843-0.910)       105        1.1        N/A
Diabetes             0.657 (0.603-0.703)       71         0.7        N/A
Atrial_Fib           0.628 (0.539-0.690)       48         0.5        N/A
CKD                  0.755 (0.688-0.862)       29         0.3        N/A
All_Cancers          0.722 (0.684-0.792)       68         0.7        N/A
Stroke               0.722 (0.560-0.813)       15         0.1        N/A
Heart_Failure        0.813 (0.665-0.878)       29         0.3        N/A
Pneumonia            0.842 (0.797-0.888)       54         0.5        N/A
COPD   

/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_9974/1440258676.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pi_offset = torch.load(f"/Users/sarahurbut/Lib

Filtering for Female: Found 5409 individuals in cohort
Filtering for Male: Found 4591 individuals in cohort

Summary of Results (1-Year Risk, Sex-Adjusted, Offset=8):
--------------------------------------------------------------------------------
Disease Group        AUC                       Events     Rate (%)   C-index   
--------------------------------------------------------------------------------
ASCVD                0.863 (0.843-0.898)       128        1.3        N/A
Diabetes             0.637 (0.582-0.684)       67         0.7        N/A
Atrial_Fib           0.699 (0.575-0.804)       36         0.4        N/A
CKD                  0.773 (0.725-0.831)       46         0.5        N/A
All_Cancers          0.733 (0.667-0.784)       70         0.7        N/A
Stroke               0.673 (0.580-0.734)       24         0.2        N/A
Heart_Failure        0.861 (0.793-0.914)       30         0.3        N/A
Pneumonia            0.847 (0.765-0.899)       59         0.6        N/A
COPD   

/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_9974/1440258676.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pi_offset = torch.load(f"/Users/sarahurbut/Lib

Filtering for Female: Found 5409 individuals in cohort
Filtering for Male: Found 4591 individuals in cohort

Summary of Results (1-Year Risk, Sex-Adjusted, Offset=9):
--------------------------------------------------------------------------------
Disease Group        AUC                       Events     Rate (%)   C-index   
--------------------------------------------------------------------------------
ASCVD                0.833 (0.781-0.861)       110        1.1        N/A
Diabetes             0.669 (0.588-0.698)       74         0.7        N/A
Atrial_Fib           0.624 (0.443-0.776)       19         0.2        N/A
CKD                  0.738 (0.718-0.795)       62         0.6        N/A
All_Cancers          0.631 (0.569-0.679)       74         0.7        N/A
Stroke               0.693 (0.565-0.840)       18         0.2        N/A
Heart_Failure        0.858 (0.778-0.920)       37         0.4        N/A
Pneumonia            0.849 (0.799-0.902)       59         0.6        N/A
COPD   

In [26]:
import pandas as pd
from evaluatetdccode import evaluate_major_diseases_wsex_with_bootstrap_dynamic_1year_different_start_end


all_results = []
for k in range(10):
    pi_offset = torch.load(f"/Users/sarahurbut/Library/CloudStorage/Dropbox-Personal/pi_enroll_fixedphi_age_offset_{k}_sex_0_10000_try2.pt")
    results = evaluate_major_diseases_wsex_with_bootstrap_dynamic_1year_different_start_end(
        pi_offset, Y_100k, E_100k, essentials['disease_names'], pce_df, 
        n_bootstraps=20, 
        follow_up_duration_years=1, 
        start_offset=k
    )
    df = pd.DataFrame(results).T.reset_index().rename(columns={'index': 'Disease'})
    df['offset'] = k
    all_results.append(df)


/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_9974/230268448.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pi_offset = torch.load(f"/Users/sarahurbut/Libr

Filtering for Female: Found 5409 individuals in cohort
Filtering for Male: Found 4591 individuals in cohort

Summary of Results (1-Year Risk, Sex-Adjusted, Offset=0):
--------------------------------------------------------------------------------
Disease Group        AUC                       Events     Rate (%)   C-index   
--------------------------------------------------------------------------------
ASCVD                0.890 (0.862-0.915)       78         0.8        N/A
Diabetes             0.754 (0.698-0.818)       33         0.3        N/A
Atrial_Fib           0.785 (0.658-0.862)       22         0.2        N/A
CKD                  0.871 (0.683-0.963)       4          0.0        N/A
All_Cancers          0.780 (0.701-0.854)       26         0.3        N/A
Stroke               0.717 (0.578-0.808)       5          0.1        N/A
Heart_Failure        0.851 (0.704-0.953)       15         0.1        N/A
Pneumonia            0.484 (0.376-0.581)       15         0.1        N/A
COPD   

/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_9974/230268448.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pi_offset = torch.load(f"/Users/sarahurbut/Libr

Filtering for Female: Found 5409 individuals in cohort
Filtering for Male: Found 4591 individuals in cohort

Summary of Results (1-Year Risk, Sex-Adjusted, Offset=1):
--------------------------------------------------------------------------------
Disease Group        AUC                       Events     Rate (%)   C-index   
--------------------------------------------------------------------------------
ASCVD                0.868 (0.822-0.907)       87         0.9        N/A
Diabetes             0.800 (0.713-0.855)       53         0.5        N/A
Atrial_Fib           0.814 (0.722-0.883)       22         0.2        N/A
CKD                  0.617 (0.495-0.707)       5          0.1        N/A
All_Cancers          0.754 (0.691-0.833)       44         0.4        N/A
Stroke               0.795 (0.424-0.924)       5          0.1        N/A
Heart_Failure        0.676 (0.331-0.837)       5          0.1        N/A
Pneumonia            0.665 (0.593-0.817)       12         0.1        N/A
COPD   

/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_9974/230268448.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pi_offset = torch.load(f"/Users/sarahurbut/Libr

...

Filtering for Female: Found 5409 individuals in cohort
Filtering for Male: Found 4591 individuals in cohort

Summary of Results (1-Year Risk, Sex-Adjusted, Offset=7):
--------------------------------------------------------------------------------
Disease Group        AUC                       Events     Rate (%)   C-index   
--------------------------------------------------------------------------------
ASCVD                0.925 (0.887-0.942)       105        1.1        N/A
Diabetes             0.820 (0.754-0.869)       71         0.7        N/A
Atrial_Fib           0.663 (0.594-0.750)       48         0.5        N/A
CKD                  0.786 (0.704-0.878)       29         0.3        N/A
All_Cancers          0.764 (0.698-0.818)       68         0.7        N/A
Stroke               0.734 (0.540-0.891)       15         0.1        N/A
Heart_Failure        0.845 (0.772-0.916)       29         0.3        N/A
Pneumonia            0.852 (0.799-0.917)       54         0.5        N/A
COPD   

/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_9974/230268448.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pi_offset = torch.load(f"/Users/sarahurbut/Libr

Filtering for Female: Found 5409 individuals in cohort
Filtering for Male: Found 4591 individuals in cohort

Summary of Results (1-Year Risk, Sex-Adjusted, Offset=8):
--------------------------------------------------------------------------------
Disease Group        AUC                       Events     Rate (%)   C-index   
--------------------------------------------------------------------------------
ASCVD                0.867 (0.840-0.907)       128        1.3        N/A
Diabetes             0.809 (0.754-0.864)       67         0.7        N/A
Atrial_Fib           0.727 (0.598-0.819)       36         0.4        N/A
CKD                  0.801 (0.724-0.871)       46         0.5        N/A
All_Cancers          0.787 (0.759-0.855)       70         0.7        N/A
Stroke               0.686 (0.639-0.837)       24         0.2        N/A
Heart_Failure        0.871 (0.786-0.946)       30         0.3        N/A
Pneumonia            0.863 (0.809-0.887)       59         0.6        N/A
COPD   

/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_9974/230268448.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pi_offset = torch.load(f"/Users/sarahurbut/Libr

Filtering for Female: Found 5409 individuals in cohort
Filtering for Male: Found 4591 individuals in cohort

Summary of Results (1-Year Risk, Sex-Adjusted, Offset=9):
--------------------------------------------------------------------------------
Disease Group        AUC                       Events     Rate (%)   C-index   
--------------------------------------------------------------------------------
ASCVD                0.840 (0.799-0.869)       110        1.1        N/A
Diabetes             0.771 (0.729-0.855)       74         0.7        N/A
Atrial_Fib           0.625 (0.439-0.776)       19         0.2        N/A
CKD                  0.757 (0.685-0.830)       62         0.6        N/A
All_Cancers          0.696 (0.648-0.744)       74         0.7        N/A
Stroke               0.697 (0.599-0.787)       18         0.2        N/A
Heart_Failure        0.877 (0.817-0.935)       37         0.4        N/A
Pneumonia            0.865 (0.768-0.922)       59         0.6        N/A
COPD   